# Getting Started with Determined, the Open-Source Deep Learning Training Platform - Lab 1
## Learning the base principles of Determined

For this part of the lab, you will consider the well-known Iris classification problem of predicting an Iris species based on the length and width measurements of their sepals and petals. You'll leverage Determined to train and tune this TensorFlow Keras based Neural Network model on TensorFlow's publicly available Iris [training](http://download.tensorflow.org/data/iris_training.csv) and [validation](http://download.tensorflow.org/data/iris_test.csv) datasets. 

The objective of the model is to predict the **likelihood** that a flower is the given Iris species.

The small dataset consists of:
* 150 samples (120 samples for the training dataset, 30 samples for the validation dataset)
* 4 features (the characteristics of the Iris): sepal length, sepal width, petal length, petal width (in cm)
* 1 label (the species of Iris to predict): for this dataset it is an integer value of 0, 1, 2 that corresponds to the species of Iris: 0-Iris setosa, 1-Iris versicolor, 2-Iris virginica.

Determined provides a Web User Interface (UI), a Command Line Interface (CLI) and APIs to interact with the Determined. In this part of the lab, you will learn how you can communicate with Determined, run tasks such as training model tasks, as well as monitor and visualize training progress and results.

* You will be introduced to Determined components.
* You will learn how to use the Determined CLI to interact with Determined.
* You will use the Determined CLI to create a training task in order to train your Neural Network model with a single GPU.
* You will also interact with the Determined AI WebUI and integrated TensorBoard to visualize experiments metrics and results.
* You will finally use the Determined's Python API to interact with Determined, load the trained model and make some predictions (inferences) using the trained model.

> <font color="green"> **Note:** The model has already been ported to run on Determined. Porting a deep learning model code to Determined is beyond the scope of this workshop. The easiest way to learn how to port your existing deep learning model code to Determined, is to start with the [Pytorch Porting tutorial](https://docs.determined.ai/latest/tutorials/pytorch-porting-tutorial.html).</font>

## 1- Authenticate to HPE Ezmeral Runtime Environment as a tenant user

For the purpose of this workshop, you will interact with the Kubernetes cluster, managed by HPE Ezmeral Runtime Enterprise, to fetch some information about the components that make up the Determined environment.

>**Note:** In normal operations, you do NOT need to interact with the Kubernetes cluster. The IT operations team would provide you with the information you need about the Determined environment to interact with it.

In order to interact with the Kubernetes cluster resources made available for your tenant, you first need to authenticate as an HPE Ezmeral tenant user of the Kubernetes Tenant (which serves as Kubernetes namespace). You will then fetch the ***kubeconfig file*** used to configure access to the resources of the Kubernetes cluster for your tenant. Here, you will accomplish these 2 steps by using the REST API of HPE Ezmeral Runtime Enterprise. 

In [ ]:
# environment variables to be verified by the student
#
studentId="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student{{ STDID }}" # your HPE ECP tenant login credentials - username 
password="{{ PASSSTU }}" # your HPE ECP tenant login credentials - password - Please replace with password provided in your registration

tenantname="DeterminedAI" #case sensitive
#
# fixed environment variables setup by the HPE ECP lab administrator - Please DO NOT MODIFY!!
#
gateway_host="{{ HPEECPGWNAME }}"
controller_endpoint="${gateway_host}:8080"
Internet_access="{{ JPHOSTEXT }}"

echo "your operation context is:" $username "on tenant" $tenantname 

#### Authenticate as a tenant user in the specified tenant

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

#### Get the Kubeconfig file for your tenant working context

In [ ]:
mkdir -p ~/.kube
ls ~/.kube
rm -f ~/.kube/config
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ~/.kube/config
#
echo "Kubeconfig file fetched. Your current Kubernetes working context is: "
kubectl config current-context

## 2- Determined components

For this hands-on workshop, Determined has been installed on the Kubernetes cluster managed by HPE Ezmeral Runtime Enterprise on a Kubernetes namespace ***determinedai***.  

When installing Determined on Kubernetes, an instance of the **Determined Master** and a **PostgreSQL database** are deployed in the Kubernetes cluster. These components run as a container within a Kubernetes POD. 

#### Run the code cell below and check out the output. 

You should see one container POD for the Determined Master service and another POD for the Database service as well as service endpoints for the Master and the database. The Master service endpoint is a NodePort service that exposes the Master service endpoint outside the Kubernetes cluster and allows you to communicate with Determined.

In [ ]:
kubectl get pod,services -n determinedai | grep determined

The **Determined Master** is the central component of the Determined deployment on Kubernetes. The Master is responsible for:

* **Scheduling** Determined training tasks as a collection of Kubernetes PODs. The Master brings up PODs to run model training tasks (known as Trials), and auxiliary tasks such as TensorBoard instances and JupyterLab instances.
* **Tracking and storing** all model training tasks metadata (description, labels, hyperparameters, search algorithm used, training metrics, validation metrics, start/end time, logs) in the PostgreSQL database.
* **Serving** the Web User Interface (WebUI) for users to visualize training metrics and validation metrics across their model training tasks.

>**Note:** _For this hands-on lab, the Kubernetes cluster worker nodes that run Determined software have been configured to connect to a distributed file system provided by HPE Ezmeral Runtime Enterprise (from the pre-integrated HPE Ezmeral Data Fabric). The distributed file system provides shared storage that works with Determined to enable:_ 
>* _training tasks launched as container PODs on any Kubernetes worker nodes to access the shared model training and validation datasets,_ 
>* _training tasks to save and store model training artifacts (model files, code, model definition files) and training task's checkpoints on a shared checkpoint storage. Checkpoints are saved versions of validated models that users can access later to test the model and deploy the model in production. Checkpoints are also used by Determined to ensure training work is not lost in case of system failure during a training, so Determine can retry failed training tasks from latest checkpoint._

**Now, let's see the Determined in action!!!**

## 3- Communicating with Determined using the Determined CLI 

The Determined CLI is a command line tool that allows you to interact with Determined. For example, the CLI allows you to launch a new experiment to train your deep learning (DL) model. The Determined CLI is distributed as a Python package. You need a machine with Python 3.6 or later installed on your machine with Internet access. The Determined CLI has already been installed on the JupyterHub server.

To use Determined and interact with Determined with the CLI, you first need to tell the CLI where the Determined Master service is running. You then need to authenticate as a user of Determined.

#### Set the Determined Master service endpoint URL:

Any Det CLI command is in the form: ***det [-m \<det_master_URL_or_IP:port\>] \<command_argument\> \<action_verb\> [-h]***

The Master service endpoint is referenced using the ***-m*** flag to specify the URL of the Determined Master that the CLI connects to. 

Instead of specifying the -m flag in every command, you can define an environmental variable: ***DET_MASTER*** that points to the Master service endpoint URL. 

>Note: You can use the help flag [-h] to learn more about valid options.


Run the code cell below to get the Determined Master endpoint URL from the Kubernetes service of Determined Master. The _"kubectl describe service"_ command is used to get the Master URL.  

In [ ]:
#
# Getting the DeterminedAI Master service endpoint URL:
#
masterUrl=$(kubectl describe service determined-master-service-stagingdetai -n determinedai | grep gateway/8080 | awk '{print $3}')
determined_master="http://${masterUrl}"
export DET_MASTER=${determined_master}
echo "The Determined Master Service endpoint URL is exported as environmental variable: " ${determined_master}

#### Authenticate to Determined:
Run the code cell below and follow steps 1 to 4 below to authenticate to Determined as student\<yourId\>: 

In [ ]:
echo "export DET_MASTER=${determined_master}" 
echo "det user login ${studentId}"
echo "---------------------------"
echo "copy/paste the two lines above to a Terminal"
echo "your password is: ${password}"

1. Start a Terminal in the Launcher (navigate to Launcher tab --> Click Terminal tile; or go to Menu --> File --> New Launcher --> Terminal)
2. In the Terminal, copy/paste the two commands above to authenticate to Determined as user Student\<yourID\>.
3. Press the _Return_ key when prompted to enter a password. Please enter the password for your studentID in Determined.
4. Then, continue from here. 

The command below displays the Determined CLI client version and Master version. 

In [ ]:
det version

## 4- Launch your first Determined training workloads to train your model

Let's first introduce some fundamental Determined concepts that are leveraged in this workshop.

Determined permits data science teams to launch deep learning model training tasks (known as **trials**) for their models ported to Determined. These tasks are distributed across one or more GPUs as an **experiment** using a particular set of configuration parameters specified in an **experiment configuration file**.

* **Experiment:** In Determined terms, an ***experiment*** is a collection of one or more DL training tasks (trials). A Determined experiment can either train a single model with a single training task using one or multiple GPUs, or it can define a search over a user-defined hyperparameter space with several training tasks.

* **Trial:** Each training task in an experiment is called a ***trial***. A trial is a training task that consists of the dataset (training and validation/test dataset), a deep learning model (for example, the Python scripts that load the dataset, build and compile the model) `adjusted to run on Determined`, and _an experiment configuration file_. All the elements of a training task are put together in a `model definition directory`.

* **Experiment configuration file:** Determined uses a YAML manifest file that defines how to run the training model process on Determined in terms of the ***hyperparameters***, the number of GPUs to use for each trial, the amount of data on which to train a model, how often the trial task must report the training metrics and the validation metrics to the Master, how often the trial task must save the model file, and many other parameters.

>**Note:** The Experiment configuration file has some required field and some optional ones. To learn more about Experiment configuration settings, check out the online documentation [here](https://docs.determined.ai/latest/training-apis/experiment-config.html).

* **Hyperparameters:** These are user-defined variables that define how a model is trained. They affect the accuracy of the trained model. By choosing the best combination of hyperparameters, you can obtain better performance for your model. 

#### Run the code cell below to display the content of the `model definition directory`:

In [ ]:
ls Code -l

The Determined model definition directory contains:
- `model_def.py`: The model definition exposed to Determined. This is the core code for the model. This includes data loading code, building the model and compiling the model.
- `*.yaml`: A set of experiment configuration YAML files that each define settings for how an experiment runs to train the model
     - _const.yaml_: Trains the model with single GPU and with constant hyperparameter values, and data located in a shared file system storage.
     - _distributed.yaml_: Same as const.yaml, but trains the model with multiple GPUs (distributed training).
     - _adaptive.yaml_: Performs a hyperparameter search using Determined's state-of-the-art adaptive hyperparameter tuning algorithm (aka a `Searcher` method).
- `startup-hook.sh`: (optional) Additional dependencies that Determined will automatically install into each POD container for this experiment. In the Iris classification example used here, Pandas Python library will be installed.

#### Let’s start simple by training the Iris deep Learning model on a single GPU by defining the hyperparameters as fixed values in the experiment configuration file. 

#### First, run the code cell below to take a closer look at the `experiment configuration file` const.yaml.

In [ ]:
cat Code/const.yaml

The ***experiment configuration file*** defines the hyperparameters, the Searcher method to use and the settings for that Searcher, the number of GPUs for each trial, the amount of data (batches or epochs) on which to train the model, how often to report training metrics and when the validation occurs, and how often to save (checkpoint) the validated model file.

As you can see above, the hyperparameters (for example the _learning_rate_ and the _batch_size_) are defined as fixed values.

As for the ***Searcher*** section, Determined used the concept of Searcher to define how Determined should explore the hyperparameter space, the amount of data on which to train the model, and the validation metric. Here, the Searcher method is defined as _Single_ because fixed values are used for the hyperparameters. In this case, Determined does not perform any hyperparameter search and optimization at all. Here, the amount of data on which to train the model is set to 5000 batches.
   * ***batch***: group of records passed to the neural network model during training. The hyperparameter ***batch_size*** defines the number of records within a batch.

The validation metric ***val_categorical_accuracy*** is used to evaluate the performance of the model and check how well the model performs on new data. For our Iris model use case, the higher the metric, the better the model as per the parameter: _smaller_is_better: false_. 

The resource setting ***slot_per_trial*** specifies the number of GPUs on which to run the experiment. The default value is 1. Here, we will use the default where a single GPU is used to train the model. 

The ***entrypoint*** tells the training task where to start running the model code.

The parameter ***min_validation_period*** instructs Determined how often to calculate the validation metrics (loss and accuracy) and influences how often to checkpoint the validated model. The validation metrics are plotted by the Master in a graph on the WebUI whenever they are calculated. _By default the validation is calculated at the trial end and the validated model is always checkpointed at the trial end._ In this example, with the periodic validation parameter _min_validation_period_ specified, the validation metric is calculated every 1000 batches and the valided model is checkpointed and saved as a file every 1000 batches **if** it is the best model. 

Notice the ***bind_mounts*** attributes: To run an experiment that uses data stored in a shared file system, _bind_mounts_ attributes are specified in the experiment configuration file to mount the directory that stores the data to the training task container POD. Here, the bind_mounts point to the shared file system path mounted on the Kubernetes cluster worker nodes by HPE Ezmeral Runtime Enterprise. 

#### Next, run the command below to create your first experiment! 

The Det command specifies the model configuration file to use and the model definition directory. Determined then schedules the training task as a Kubernetes POD. The POD container has all the libraries and dependencies required for training a typical deep learning model with common deep learning frameworks like PyTorch, TensorFlow, and Keras.

* _det experiment create \<experiment_config_file\> \<model_definition_directory\>_

The command will return the Experiment Id.

In [ ]:
# launch experiment to train a single model on a single GPU
det experiment create Code/const.yaml Code

Using the command below, you will see that the Determined Master has launched **one** training task (trial) for your experiment as a container POD with name in the form:

 _exp-\<experimentID\>-trial-\<TriaID\>-\<unique-name\>_
 
 >**Note:** Since this experiment trains a single model with a fixed set of hyperparameters, there is only one training task (trial) launched.
 
> <font color="blue"> **Note:** As you are sharing the same Kubernetes resources with other participants, and depending on the number of concurrent experiments running, your training task POD might be in **Pending** state waiting for GPU resources to become available. You might need to wait a few minutes until other experiments complete for your training task POD to become **Running**.</font>

In [ ]:
kubectl get pods -n determinedai

The Det commands below are used to list your most recent experiment and its status in Determined:

* _det experiment list | tail -1_
* _det experiment describe \<experiment_Id\> --json | jq .[0].state_

#### Run the code cell below to track the execution progress of your experiment.

In [ ]:
det experiment list | tail -1
# Get the experiment Id, remove spaces
myexpId=$(det experiment list | tail -1 | cut -d'|' -f 1 |  tr -d ' ')
#det experiment describe ${myexpId} --json | jq .[0].state

## 5- Monitor and visualize your experiment in Determined AI Web User Interface

To access information on both training and validation performance, you can also simply go to the Determined **WebUI** by entering the service endpoint URL of the Determined Master in your web browser connected to the Internet.

* Run the code cell below to get the Determined Master WebUI URL. 
* Then, click on the displayed link to connect. This will open a new tab in your browser with the Determined UI login banner.
* You will be prompted to enter your credentials. Type your StudentID as credentials and press return. The password is `blank` by default.
* Upon login you should see the WebUI **dashboard** as shown in the picture below. The Dashboard page shows an overview of tasks on the Determined system as well as an overview of the GPU resources utilization. 

In [ ]:
portUI=$(kubectl describe service determined-master-service-stagingdetai -n determinedai | grep gateway/8080 | awk '{print $3}' | cut -d':' -f 2 |  tr -d ' ')
##portUI = str(port)[2:-2]
echo "The Determined Master WebUI URL is: http://${Internet_access}:${portUI}"
echo "Click the link above to connect. Login using your student Identifier: ${studentId}. You password is ${password}. Click on Sign In button"

<img src="Pictures/DetWebUI-Login.png" height="298" width="300">

From the WebUI, make sure **you select your StudentID** from the ***Users*** drop-down list as shown in the picture below. By default, the Experiments are displayed. You can select other icons to display auxiliary tasks such as TensorBoard tasks and JupyterLab tasks. We will explore these auxiliary tasks in the next sections. 

<img src="Pictures/DetWebUI-Users-v1.png" height="171" width="900">


##### From the dashboard, you should see the experiment as an **ACTIVE** state and its completion percentage, or as **COMPLETED**.

> <font color="blue"> **Important Note:** If there are multiple concurrent participants to the workshop, your experiment might not run yet because there are more experiments running than the Kubernetes cluster has GPUs. You might need to wait a few minutes until other experiments complete for your experiment to start running. </font>

#### Select the most recent experiment you want to visualize.

As the experiment runs, the graph is showing the model **validation** accuracy metric (_val_categorical_accuracy_) over the number of completed batches. You can see the graph changing in real time as the experiment runs.

From the **Metrics** menu, under **Training Metrics**, select _categorical_accuracy_ (see picture below for an example). This metric indicates the model accuracy on **training** data while the _val_categorical_accuracy_ indicates the model accuracy on **validation** data. 

<img src="Pictures/WebUI-Exp-const-Metrics-selection.png" height="297" width="700">

As you can see in the graphs, the Master plots training metrics every **100 batches** of training data by default, while the validation metrics ("validation" accuracy) are plotted every 1000 batches based on the experiment configuration parameter _min_validation_period_.

After the experiment completes, although the validation accuracy may differ from one trial to another, you can see on the experiment detail page that training the model with the hyperparameter settings in `const.yaml` yields a validation accuracy between 93% and 97%. 

Scroll down to see a list of training validation workloads and their metrics for the metric types you previously selected. 
You might see one or two validation workloads with checkpoints as shown in the picture below. With the default checkpoint collection policy, Determined will checkpoint the most recent validated model and the best model per training task (trial). If the most recent checkpoint is also the best checkpoint for a given trial, only one checkpoint will be saved for that trial. 

<img src="Pictures/WebUI-Exp-const-graph.png" height="520" width="900">

## 6 - Launch a TensorBoard visualization instance as **auxiliary workload** on Determined

[TensorBoard](https://www.tensorflow.org/tensorboard) is a widely used tool by ML engineers for visualizing and inspecting the learning curve of the trained deep learning models. Determined is integrated with TensorBoard for deeper analysis of your experiment and to help you examine your neural network model by viewing the training and validation loss curves for your experiment in TensorBoard. 

Determined lets you launch **auxiliary workload** such as a Tensorboard server, and access TensorBoard in one-click from the WebUI, or you can run the following command in Determined’s command line:

* _det tensorboard start \<experiment_Id\>_

#### Run the code cell below to launch the TensorBoard server instance.

This may take a minute or so as Determined has to launch the Tensorboard server as a Kubernetes POD. 

In [ ]:
echo "Start a Tensordboard server instance for your Experiment ${myexpId} with TensorBoard instance ID:"
# start the tensorBoard server instance for the experiment
det tensorboard start -d ${myexpId}

>**Note:** Run the code cell below. You will notice that the TensorBoard server instance is launched as a container POD in the Kubernetes cluster. The POD name is in the form "_cmd-0-\<TensorBoard-Instance-ID\>_". Determined proxies HTTP requests to and from the TensorBoard container through the Determined Master node.

In [ ]:
kubectl get pod -n determinedai

#### Once the POD is in running state, run the code cell below to get the Tensorboard URL for your experiment. Then, click on the link to connect.
>Note: It may take a minute for the POD to be in running state. Please wait one minute to continue.

In [ ]:
mytensorboard=$(det tensorboard list | grep RUNNING | cut -d'|' -f 1 |  tr -d ' ')
##mytensorboard=str(mytensorboard)[2:-2]
#print (f"{mytensorboard}")
echo "Your tensorboard is running at http://${Internet_access}:${portUI}/proxy/${mytensorboard}/"
echo "Click on the link to connect."

<img src="Pictures/TensorBoard-const-graph.png" height="413" width="900">

Determined created TensorBoard plots to show the training loss, validation loss, training accuracy and validation accuracy for the training task (trial).

#### When you have finished with Tensorboard, run the code cell below to `kill` the Tensorboard process

In [ ]:
det tensorboard kill ${mytensorboard}

## 7 - List the best model created by the training process
By default, Determined will save the most recent and the best checkpoint per training task (trial) according to the validation metrics specified in the Searcher section of the configuration file for the experiment.

* _det experiment list-checkpoints [--best] [N best checkpoints to return] \<experiment_Id\>_

>**Note**: Upon completion of the training task, if the most recent checkpoint is also the best checkpoint for a given trial, only one checkpoint will be saved for that trial by Determined. Otherwise, two checkpoints will be saved. Other checkpoints will be automatically deleted to reclaim space. You can verify this using the command below.

#### Run the code cell below to display the best checkpoint(s) for your experiment

In [ ]:
#list the best Trial checkpoint(s) (training task):
det experiment list-checkpoints --best 2 ${myexpId}

## 8 - Launch a JupyterLab instance as **auxiliary workload** on Determined

Users can also launch a JupyterLab server instance as **auxiliary workload** on Determined, in which they run Jupyter Notebooks. This is useful to load and test a model that was trained during the experiment because the Determined CLI is installed into the JupyterLab server instance by default, and the JupyterLab server container has access to the shared file system where the checkpoints are stored. 

In the next section of this part of the lab, you can use the JupyterLab server instance on Determined to test your trained model and make predictions. 

Determined lets you launch an instance of a JupyterLab server in Determined in Kubernetes and access the JupyterLab server in one-click from the WebUI, or you can run the following command in Determined’s command line:

* _det notebook start [--config-file configurationFile]_

The configuration file is used to control aspects of the JupyterLab environment such as a description, the checkpoint volume where trial checkpoints are stored in the shared file system, and the resources (CPU or GPU) used to launch the JupyterLab server. Run the next code cell to look at the content of the configuration file.

#### Run the code cell below to examine the settings for the JupyterLab instance.
The configuration file used here allows you to launch a JupyterLab server instance that does not use any GPUs (***resources.slot=0***) and that gets access to the shared checkpoint storage area where the model artifacts are stored. The shared checkpoint storage is mounted to ***/determined_shared_fs*** mount point inside Jupyterlab container POD. 

In [ ]:
cat Code/notebook-config.yaml

#### Run the code cell below to launch an instance of the JupyterLab
It may take a minute or so for the JupyterLab instance to become active as Determined has to launch the JupyterLab server instance as a Kubernetes POD in the Kubernetes cluster. 

In [ ]:
echo "Start a JupyterLab server instance within Determined system with instance ID:"
# start the Jyputer Notebook server instance for the experiment
det notebook start -d --config-file Code/notebook-config.yaml

>**Note:** Run the code cell below. You will notice that JupyterLab instance is launched as a container POD in the Kubernetes cluster. The POD name is in the form "_cmd-0-\<Jupyterlab-Instance-ID\>_". Determined proxies HTTP requests to and from the JupyterLab container through the Determined Master node.

In [ ]:
kubectl get pod -n determinedai

#### Check the status of the JupyterLab instance using the command below:
* _det notebook list_

In [ ]:
det notebook list | grep -e RUNNING -e STARTING

## 9- Inferences with Determined
When you train a model with Determined, all of the artifacts (model files) associated with that training tasks are tracked and stored in the _checkpoint storage_. Determined lets you access the artifacts programmatically using the Python API from within the JupyterLab server launched on Determined system. This makes it really easy for you to export your best-performing trained model out of Determined and load it for testing the model by making **inferences** (the process of using a trained model and new unlabeled data to make a prediction).

* More information about the Determined's Python API can be found [here](https://docs.determined.ai/latest/interact/api-experimental-client.html).
* More information for downloading a trained model can be found [here](https://docs.determined.ai/latest/post-training/use-trained-models.html).

#### Run the code cell to adjust some environment variables in the notebook **Inferences.ipynb**

In [ ]:
sed -i "s/USERNAME/${studentId}/" Inferences.ipynb
sed -i "s/EXPID/${myexpId}/" Inferences.ipynb
sed -i "s/MASTERURL/${masterUrl}/" Inferences.ipynb
sed -i "s/PASSW/${password}/" Inferences.ipynb

#### Next, download the file **Inferences.ipynb** to your local PC/laptop. 

You will use this notebook to test your trained model by making some inferences from JupyterLab instance you have just launched on Determined.

Right-click on the file **Inferences.ipynb** and choose **Download**.

#### Now, connect to the JupyterLab server instance you have just deployed: 

* Run the code cell below to get the JupyterLab URL. Then, click on the link to connect to the JupyterLab instance you have just launched on Determined.

* On the JupyterLab instance, click the ***up arrow*** icon to **upload** the file _Inferences.ipynb_ from your local PC/laptop. Once the file is uploaded, double-click the file to open the notebook. 

In [ ]:
myNotebook=$(det notebook list | grep RUNNING | cut -d'|' -f 1 |  tr -d ' ')
##echo "${myNotebook}"
echo "Your JupyterLab instance is running at http://${Internet_access}:${portUI}/proxy/${myNotebook}/"
echo "Click on the link to connect to the JupyterLab instance you just launched."
echo "On JupyterLab instance, click the up arrow to upload the file Inferences.ipynb."

> <font color="red"> **IMPORTANT: When you have finished with the Inferences in JupyterLab on the Determined, please go back to your local Jupyter Notebook to run the code cells below and perform some cleanup** </font>.

## 10- Time to cleanup: Delete the checkpoints for your experiment to reclaim storage space in the shared file system and stop the JupyterLab instance

The ***save_experiment_best***, ***save_trial_best*** and ***save_trial_latest*** parameters of the checkpoint collection policy specify which checkpoints to save. The default policy is set as follows:

  * save_experiment_best:0 
  * save_trial_best:1
  * save_trial_latest:1
 
The default **checkpoint garbage collection policy** dictates Determined to checkpoint the most recent (the latest) validated model and the best model per training task (trial). The “best” checkpoint for a trial is the checkpoint with the model judged best based on the validation metric defined in the Searcher settings of the experiment configuration file.
  
#### Run the code cell below to reclaim some storage disk space by changing the default checkpoint garbage collection policy, as shown below:

In [ ]:
# Delete the checkpoints data for the single model training using a single GPU
det experiment set gc-policy --yes --save-experiment-best 0 --save-trial-best 0 --save-trial-latest 0 ${myexpId}

#### Next, delete the instance of the JupyterLab server.

In [ ]:
det notebook kill ${myNotebook}

#### Now that you have the base principles about Determined in mind, let's explore Distributing training with Determined.

Click on Lab 2 below to open a notebook to explore Distributed Training with Determined. 
* [Lab 2](2-WKSHP-DET-AI-101-Getting-started-Dist-Training.ipynb)